In [19]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Configuration et Chargement du Modèle ---

# Charger le modèle ResNet50 pré-entraîné sur ImageNet
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=True)
model.trainable = False # Geler le modèle, nous n'optimisons que le masque

# Définir la taille d'entrée attendue par ResNet50
IMG_SIZE = (224, 224)

# --- 2. Chargement et Préparation de l'Image (x) ---

# Télécharger une image d'exemple
img_url = "data/chat_roux.jpg"

# Charger l'image et la redimensionner
img = tf.keras.utils.load_img(img_url, target_size=IMG_SIZE)
# 'x' est notre image de base (0-255), de forme (224, 224, 3)
x = tf.keras.utils.img_to_array(img)

# Fonction pour pré-traiter l'image pour ResNet50
def preprocess_image(img_tensor):
    # Ajoute la dimension batch et applique le pré-traitement ResNet
    return tf.keras.applications.resnet50.preprocess_input(img_tensor[tf.newaxis, ...])

# --- 3. Obtenir la Prédiction Originale (Phi(x)) ---

# Pré-traiter l'image originale
x_preprocessed = preprocess_image(x)

# Obtenir les prédictions (logits)
original_preds = model(x_preprocessed, training=False)

# Appliquer Softmax pour obtenir les probabilités [cite: 161]
original_probs = tf.nn.softmax(original_preds[0])

# Obtenir la classe prédite (j*) et sa probabilité
j_star = tf.argmax(original_probs)

# Afficher les 5 prédictions
decoded_preds = tf.keras.applications.resnet50.decode_predictions(original_preds.numpy(), top=5)
original_prob_value = decoded_preds[0][0][2]
print("Les 5 meilleures prédictions pour l'image :")
for i in range(5):
    class_name = decoded_preds[0][i][1]
    prob = decoded_preds[0][i][2]
    print(f"  {i+1}. {class_name} : {prob:.4f}")

# Afficher la prédiction principale qui sera utilisée pour RDE
print(f"\nPrédiction principale (utilisée pour RDE) : {decoded_preds[0][0][1]} (Classe {j_star}) avec probabilité {original_prob_value:.4f}")

Les 5 meilleures prédictions pour l'image :
  1. Egyptian_cat : 0.3862
  2. tabby : 0.2892
  3. tiger_cat : 0.1696
  4. printer : 0.0292
  5. hamper : 0.0179

Prédiction principale (utilisée pour RDE) : Egyptian_cat (Classe 285) avec probabilité 0.3862


In [20]:
# Hyperparamètres
lambda_val = 0.6        # 
num_steps = 2000        # 
num_samples = 64        # [cite: 239]
optimizer = tf.keras.optimizers.Adam(learning_rate=0.003) # 

# Le masque entraînable
s = tf.Variable(tf.ones_like(x), trainable=True)

# Moyenne et écart-type pour le bruit v 
mu = tf.reduce_mean(x)
sigma = tf.math.reduce_std(x)

for step in range(num_steps):
    
    with tf.GradientTape() as tape:
        # 1. Garder le masque s dans [0, 1]
        # (Nous le faisons via assign pour que la variable 's' soit mise à jour)
        s.assign(tf.clip_by_value(s, 0, 1)) 
        
        # 2. Estimer l'espérance de la distorsion (Monte-Carlo)
        distortion = 0.0
        for _ in range(num_samples):
            # Générer le bruit v [cite: 146, 236]
            v = tf.random.normal(shape=x.shape, mean=mu, stddev=sigma, dtype=tf.float32)
            
            # Créer l'entrée perturbée y
            y = x * s + (1 - s) * v
            
            # Obtenir la prédiction perturbée Phi(y)
            # Ajout de la dimension batch (batch_size=1)
            y_batch = y[tf.newaxis, ...] 
            perturbed_output = model(y_batch, training=False) # training=False est important
            
            # Obtenir le score pour la classe d'origine j*
            perturbed_prob = perturbed_output[0, j_star]
            
            # Calculer la distorsion d (ex: d_2 )
            # L'article multiplie par 100, mais c'est juste une mise à l'échelle
            distortion += tf.square(original_prob_value - perturbed_prob)
            
        # Moyenne de la distorsion
        D = distortion / num_samples
        
        # 3. Calculer la pénalité de parcimonie (R) [cite: 24]
        R = lambda_val * tf.norm(s, ord=1) # Norme L1
        
        # 4. Perte totale
        total_loss = D + R

    # 5. Rétropropagation pour mettre à jour le masque 's'
    # Calcule les gradients de la perte par rapport à 's'
    gradients = tape.gradient(total_loss, [s])
    
    # Applique les gradients à 's'
    optimizer.apply_gradients(zip(gradients, [s]))

# 6. Le résultat final est le masque 's' optimisé
final_explanation_mask = s.numpy() # Convertit en array NumPy

AbortedError: {{function_node __wrapped____MklNativeConv2DBackpropInput_device_/job:localhost/replica:0/task:0/device:CPU:0}} Operation received an exception:Status: 1, message: could not create a memory object, in file tensorflow/core/kernels/mkl/mkl_conv_grad_input_ops.cc:546 [Op:Conv2DBackpropInput] name: 